# Operating tables

This section focuses on creating, fulfilling, and analyzing tables in PostgreSQL.

For more infomation check:

- [Manual on `CREATE TABLE`](https://www.postgresql.org/docs/current/sql-createtable.html) directive in postgreSQL.

In [1]:
docker run -d --rm\
    --name building_tables_examples_pg\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4

sleep 5

b8131450f737d8553034038439d827b756ff40f9735fb7134d7b9a05f993cb11


**Note**: Don't forget to stop the experimental container afterward.

In [33]:
docker stop building_tables_examples_pg

building_tables_examples_pg


## Default values

You can define default values that will be used during table creation. You need to use the `DEFAULT` keyword after specifying the column type, followed by the value that the column should take.

An important practical nuance is that you can use expressions as values for DEFAULT. These are used when new table records are created.

Check more:

- [Page "Default Values"](https://www.postgresql.org/docs/current/ddl-default.html) on the official postgreSQL manual. 

---

In the following example, a table is created that defines `DEFAULT` values for columns of different data types:

In the following example, a table is created with columns of different data types, and `DEFAULT` values are assigned to each. Specifically:

- For `BOOL`, `REAL` and `JSONB` data types, **literal values** are used as defaults.
- For `TIMESTAMP`, the **`NOW()` function** is used as a default expression, which returns the current timestamp at the moment of insertion.

In [5]:
docker exec -i building_tables_examples_pg psql -U postgres << EOF
DROP TABLE IF EXISTS default_values_example;
CREATE TABLE default_values_example (
    bool_value BOOL DEFAULT True,
    real_value REAL DEFAULT 10,
    date_time TIMESTAMP WITHOUT TIME ZONE DEFAULT NOW(),
    json_type JSONB DEFAULT '{"A": 10, "B": 20}'
)
EOF

DROP TABLE
NOTICE:  table "default_values_example" does not exist, skipping
CREATE TABLE


The follwing cell shows inserting to the tables procedure. With sepecifying `DEFAULT` instead of columns names you can create row with totaly default values.

In [6]:
docker exec -i building_tables_examples_pg psql -U postgres << EOF
INSERT INTO default_values_example DEFAULT VALUES;
EOF

INSERT 0 1


Or you can specify only some of the columns; the others will use their default values:

In [7]:
docker exec -i building_tables_examples_pg psql -U postgres << EOF
INSERT INTO default_values_example(bool_value, real_value) 
VALUES
    (false, 30),
    (false, 32);
EOF

INSERT 0 2


The following cell displays the table that we obtained at the end.

In [8]:
docker exec -i building_tables_examples_pg psql -U postgres << EOF
SELECT * FROM default_values_example;
EOF

 bool_value | real_value |         date_time          |     json_type      
------------+------------+----------------------------+--------------------
 t          |         10 | 2024-10-19 16:47:41.397551 | {"A": 10, "B": 20}
 f          |         30 | 2024-10-19 16:47:43.031244 | {"A": 10, "B": 20}
 f          |         32 | 2024-10-19 16:47:43.031244 | {"A": 10, "B": 20}
(3 rows)



## Loading from csv

This section shows how data may be loaded from `csv` file. 

Learn more in:

- [Import CSV file into PostgreSQL table](https://www.postgresqltutorial.com/postgresql-tutorial/import-csv-file-into-posgresql-table/) tutorial.
- [COPY command](https://www.postgresql.org/docs/current/sql-copy.html) in official postgres documentation page. 

You can specify file in the `FROM` section `DELIMITER` and `CSV` parameters specify how file should be loaded.

---

The following cell creates a CSV file in the container and reads it into the database.

In [6]:
docker exec -i building_tables_examples_pg sh -c "cat > /test.csv" << EOF
20,Riga
30,Vilnus
4,Minsk
43,Warsaw
EOF


docker exec -i building_tables_examples_pg psql -U postgres << EOF
-- creating table
DROP TABLE IF EXISTS csv_example;
CREATE TABLE csv_example (
    value REAL, city VARCHAR(20)
);

-- loading csv table to created table
COPY csv_example(value, city)
FROM '/test.csv'
DELIMITER ','
CSV;
EOF

NOTICE:  table "csv_example" does not exist, skipping
DROP TABLE
CREATE TABLE
COPY 4


Now let's check the obtained results:

In [7]:
docker exec -i building_tables_examples_pg psql -U postgres << EOF
SELECT * FROM csv_example LIMIT 10;
EOF

 value |  city  
-------+--------
    20 | Riga
    30 | Vilnus
     4 | Minsk
    43 | Warsaw
(4 rows)



## Indexes

Indexing in PostgreSQL is a way to increase the performance of queries, but it adds overhead to the database system as a whole.

For more information, check the [corresponding chapter](https://www.postgresql.org/docs/current/indexes.html) in the PostgreSQL documentation.

---

The following cell creates a table that contains a relatively large number of rows. An index is added to one of the columns (`indexed_id`).

In [24]:
docker exec -i building_tables_examples_pg psql -U postgres << EOF
DROP TABLE IF EXISTS indexes_example;
CREATE TABLE indexes_example (just_id integer, indexed_id integer, content varchar);

INSERT INTO 
    indexes_example (just_id, indexed_id, content)
SELECT
    (random()*100)::INT,
    (random()*100)::INT,
    chr(65 + floor(random()*26)::INT)
FROM
    generate_series(1, 1000000);

CREATE INDEX test_index ON indexes_example (indexed_id);

EOF

DROP TABLE
CREATE TABLE
INSERT 0 1000000
CREATE INDEX


Now let's compare the performance of the query that adds conditions under the `just_id` column.

In [31]:
docker exec -i building_tables_examples_pg psql -U postgres << EOF
EXPLAIN ANALYZE SELECT * FROM indexes_example WHERE just_id=50;
EOF

                                                             QUERY PLAN                                                             
------------------------------------------------------------------------------------------------------------------------------------
 Gather  (cost=1000.00..12627.63 rows=10133 width=10) (actual time=0.153..13.787 rows=10067 loops=1)
   Workers Planned: 2
   Workers Launched: 2
   ->  Parallel Seq Scan on indexes_example  (cost=0.00..10614.33 rows=4222 width=10) (actual time=0.011..11.004 rows=3356 loops=3)
         Filter: (just_id = 50)
         Rows Removed by Filter: 329978
 Planning Time: 0.147 ms
 Execution Time: 13.981 ms
(8 rows)



And the query that adds conditions under the `indexed_id` column.

In [32]:
docker exec -i building_tables_examples_pg psql -U postgres << EOF
EXPLAIN ANALYZE SELECT * FROM indexes_example WHERE indexed_id=50;
EOF

                                                          QUERY PLAN                                                           
-------------------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on indexes_example  (cost=123.61..5673.19 rows=10733 width=10) (actual time=1.091..4.763 rows=10178 loops=1)
   Recheck Cond: (indexed_id = 50)
   Heap Blocks: exact=4593
   ->  Bitmap Index Scan on test_index  (cost=0.00..120.92 rows=10733 width=0) (actual time=0.647..0.647 rows=10178 loops=1)
         Index Cond: (indexed_id = 50)
 Planning Time: 0.183 ms
 Execution Time: 4.939 ms
(7 rows)



The second query typically executes faster.

## Tables properties